<a href="https://colab.research.google.com/github/Athugodage/legal_dtm/blob/main/BERTopic_DTM_legal_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the dataset

In [5]:
##  Клонируем репозиторий с датасетом

!git clone https://github.com/Athugodage/legal_dtm.git

Cloning into 'legal_dtm'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [6]:
!unzip '/content/legal_dtm/unfiltered_corpus.zip'

Archive:  /content/legal_dtm/unfiltered_corpus.zip
  inflating: unfiltered_corpus.csv   


In [7]:
import pandas as pd


df = pd.read_csv('unfiltered_corpus.csv')

In [8]:
df

##  сам датасет
##  мы будем использовать только колонки Текст и Дата (которую преобразуем в год)

,Название документа,Ссылка,Текст,Комментарий РГ,Дата
0,Указ временно исполняющего обязанности главы Д...,https://rg.ru/documents/2022/11/28/dnr-ukaz37-...,1. Внести в Указ временно исполняющего обязанн...,Врио главы ДНР Денис Пушилин подписал указ об ...,28 ноября 2022
1,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации Конституционный С...,"Бессрочный запрет на профессию для адвокатов, ...",25 ноября 2022
2,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,25 ноября 2022
3,Федеральный закон от 21 ноября 2022 г. N 443-Ф...,https://rg.ru/documents/2022/11/24/document-na...,Принят Государственной Думой 10 ноября 2022 го...,На сладкую газировку с 1 июля 2023 года введут...,24 ноября 2022
4,Федеральный закон от 21 ноября 2022 г. N 448-Ф...,https://rg.ru/documents/2022/11/24/document-by...,Принят Государственной Думой 10 ноября 2022 го...,Приостановленное в этом году бюджетное правило...,24 ноября 2022
...,...,...,...,...,...
2958,Постановление о совершенствовании госрегулиров...,https://rg.ru/2009/08/14/lekarstva-dok.html,Правительство Российской Федерации постановляе...,Производители жизненно необходимых и важнейших...,14 августа 2009 г.
2959,"Правила оплаты проезда осужденных, самостоятел...",https://rg.ru/2009/08/12/pravila-dok.html,В соответствии с частью второй статьи 751 Угол...,"Правительство утвердило новые правила, как опл...",12 августа 2009 г.
2960,Изменения в постановление правительства о него...,https://rg.ru/2009/08/07/sysk-izmenenia-dok.html,Правительство Российской Федерации постановляе...,Частные охранники и детективы должны будут про...,7 августа 2009 г.
2961,Постановление Главного госсанврача о надзоре з...,https://rg.ru/2009/08/05/bad-dok.html,Зарегистрирован в Минюсте РФ 10 июля 2009 г. ...,В России усилят контроль за производством и об...,5 августа 2009 г.


# **Tutorial** - Dynamic Topic Modeling with Trump's Tweets
(last updated 11-09-2022)

In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Trump's Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [9]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data Preprocessing**
For this tutorial, we will be needing to extract all Trump's Tweet from his @realDonalTrump account. We will be removing all retweet and focus on his original tweets. 

Moreover, since we are looking at his tweets over time, we will be saving all timestamps related to his tweets.

In [10]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 835.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9b602ed5282d7017c28b59b698bd8c62839ec61bb5783a971972a9e869a828b1
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [11]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [12]:
from tqdm import tqdm

tqdm.pandas()

stopwords = open('/content/legal_dtm/russian')

def lemmatize(f):
    
    text = f.split()
    txt = []
    for word in text:
        if word not in stopwords:
            word = morph.parse(word)[0].normal_form
            txt.append(word)
    doc = ' '.join(txt)

    return doc

df['Lemmatized_text'] = df['Текст'].progress_apply(lemmatize)

##  Tokenizing and lemmatizing all words, deleting stopwords

100%|██████████| 2963/2963 [47:25<00:00,  1.04it/s]


In [13]:
import re

def month2int(string):
    months = {'января': '01',
            'февраля': '02',
            'марта': '03',
            'апреля': '04',
            'мая': '05',
            'июня': '06',
            'июля': '07',
            'августа': '08',
            'сентября': '09',
            'октября': '10',
            'ноября': '11',
            'декабря': '12'}

    day = re.search('\d{2}', string).group(0)
    month = re.search('[А-я]+', string).group(0)
    year = re.search('\d{4}', string).group(0)
    return f"{year}-{months[month]}-{day}"


df['Date'] = df['Дата'].apply(month2int)

##  transforming dates into necessary format:
##  year-month-day

In [14]:
df

,Название документа,Ссылка,Текст,Комментарий РГ,Дата,Lemmatized_text,Date
0,Указ временно исполняющего обязанности главы Д...,https://rg.ru/documents/2022/11/28/dnr-ukaz37-...,1. Внести в Указ временно исполняющего обязанн...,Врио главы ДНР Денис Пушилин подписал указ об ...,28 ноября 2022,1. внести в указ временно исполнять обязанност...,2022-11-28
1,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации Конституционный С...,"Бессрочный запрет на профессию для адвокатов, ...",25 ноября 2022,имя российский федерация конституционный суд р...,2022-11-25
2,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,25 ноября 2022,в цель обеспечение единство практика применени...,2022-11-25
3,Федеральный закон от 21 ноября 2022 г. N 443-Ф...,https://rg.ru/documents/2022/11/24/document-na...,Принят Государственной Думой 10 ноября 2022 го...,На сладкую газировку с 1 июля 2023 года введут...,24 ноября 2022,принять государственный дума 10 ноябрь 2022 го...,2022-11-24
4,Федеральный закон от 21 ноября 2022 г. N 448-Ф...,https://rg.ru/documents/2022/11/24/document-by...,Принят Государственной Думой 10 ноября 2022 го...,Приостановленное в этом году бюджетное правило...,24 ноября 2022,принять государственный дума 10 ноябрь 2022 го...,2022-11-24
...,...,...,...,...,...,...,...
2958,Постановление о совершенствовании госрегулиров...,https://rg.ru/2009/08/14/lekarstva-dok.html,Правительство Российской Федерации постановляе...,Производители жизненно необходимых и важнейших...,14 августа 2009 г.,правительство российский федерация постановляе...,2009-08-14
2959,"Правила оплаты проезда осужденных, самостоятел...",https://rg.ru/2009/08/12/pravila-dok.html,В соответствии с частью второй статьи 751 Угол...,"Правительство утвердило новые правила, как опл...",12 августа 2009 г.,в соответствие с часть второй статья 751 уголо...,2009-08-12
2960,Изменения в постановление правительства о него...,https://rg.ru/2009/08/07/sysk-izmenenia-dok.html,Правительство Российской Федерации постановляе...,Частные охранники и детективы должны будут про...,7 августа 2009 г.,правительство российский федерация постановляе...,2009-08-20
2961,Постановление Главного госсанврача о надзоре з...,https://rg.ru/2009/08/05/bad-dok.html,Зарегистрирован в Минюсте РФ 10 июля 2009 г. ...,В России усилят контроль за производством и об...,5 августа 2009 г.,зарегистрировать в минюст рф 10 июль 2009 г. р...,2009-08-20


In [15]:
df = df.sort_values(by='Date', ascending=False)

In [16]:
timestamps = df['Date'].to_list()

##  transforming DATE column into Python list

In [17]:
import re
import pandas as pd
from datetime import datetime


text = df['Lemmatized_text'].to_list()


##  transforming a column with lemmatized legal documents into Python list

# **Dynamic Topic Modeling**


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words=stopwords, min_df=1, max_df=0.6)

## training vectorizer model

## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [19]:
from bertopic import BERTopic
topic_model_1 = BERTopic(min_topic_size=7, 
                       verbose=True, 
                       language='russian', 
                       top_n_words = 10,
                       n_gram_range=(3, 5),
                       vectorizer_model=vectorizer_model)

In [20]:
topics, _ = topic_model_1.fit_transform(text)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/93 [00:00<?, ?it/s]

2023-01-08 10:31:02,388 - BERTopic - Transformed documents to Embeddings
2023-01-08 10:31:27,233 - BERTopic - Reduced dimensionality
2023-01-08 10:31:27,450 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [21]:
freq = topic_model_1.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,610,-1_избирательный_таможенный_список_земельный
1,0,105,0_лишение_свобода_наказываться_осудить
2,1,54,1_кадровый_подписания_резерв_83
3,2,47,2_коронавирусный_инфекция_covid_московский
4,3,41,3_волгоградский_коронавирусный_чрезвычайный_ин...
5,4,40,4_подданства_гражданство_проживание_постоянный
6,5,39,5_кадастровый_земельный_недвижимости_участок
7,6,39,6_образовательный_лекарственный_принудительный...
8,7,38,7_аналитический_дебет_вступительный_отражаться
9,8,38,8_платёжный_энергетический_радиоактивный_коопе...


In [22]:
topic_model_1.get_topic(4)

[('подданства', 0.11333341844992588),
 ('гражданство', 0.11254564722937696),
 ('проживание', 0.07531481356740319),
 ('постоянный', 0.07399679054672335),
 ('государства', 0.07228177425745891),
 ('охотничий', 0.06166602287323772),
 ('воинский', 0.05002222074457321),
 ('оружие', 0.03763291594498238),
 ('оружия', 0.037527786398959005),
 ('ношение', 0.03005210994825059)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [23]:
fig = topic_model_1.visualize_topics(); fig

In [24]:
topic_model_1.visualize_barchart()

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [25]:
topics_over_time = topic_model_1.topics_over_time(docs=text, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True)

882it [01:58,  7.44it/s]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [26]:
topic_model_1.visualize_topics_over_time(topics_over_time, top_n_topics=12)

## getting first 13 global topics

In [ ]:
topic_model_1.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## getting first 20 global topics